In [ ]:
from IPython import display
#display.Image("tt_comparison.png")

# Query, explore and analize data hosted on the PMD-C

- Data on the triplestore hosted on the PMD-C is explored via sparql-queries (SPARQLWrapper module)
- The data-source (raw stress and strain in csv format) is obtained from the triple store via sparql queries
- Based on the data, mechanical quantities are calculated: $E$-modulus, $R_{p0.2}$


Importing relevant packages 

In [ ]:
import os
import matplotlib.pylab as plt
from SPARQLWrapper import SPARQLWrapper, JSON
from tabulate import tabulate
import pandas as pd

## Exploring the data via sparql-queries

Here, we show how relevant data for an analysis of the stress-strain data is obtained "manually" via SPARQL-queries.

- Definition of the SPARQL endpoint, return-format and setting an API token for access authorisation:

In [ ]:
# define the spaerql endpoint and get a SPARQLWrapper object
with open('./endpoint+token.txt', 'r') as f:
    lines = f.readlines()

SPARQL_ENDPOINT = lines[0].replace('\n','')
TOKEN = lines[1].replace('\n','')

sparql = SPARQLWrapper(SPARQL_ENDPOINT)
sparql.setReturnFormat("json")

# API key from Ontodocker
sparql.addCustomHttpHeader(httpHeaderName="Authorization", httpHeaderValue=TOKEN)

- Query all uri for data of type "Tensile test"

In [ ]:
sparql.setQuery("""
PREFIX pmd: <https://w3id.org/pmd/co/>
SELECT distinct ?p
WHERE {
?s a pmd:TestPiece .
?p a pmd:TensileTest .
} ORDER BY ?p
"""
)

ret = sparql.queryAndConvert()
for r in ret["results"]["bindings"]:
    print(r['p']['value'])

- select one of these uris for the following examples:

In [ ]:
example_uri = ret['results']['bindings'][0]['p']['value']
print(example_uri)

- get all Metadata for the example uri:

In [ ]:
sparql.setQuery("""
PREFIX pmd: <https://w3id.org/pmd/co/>
SELECT distinct ?p ?o
WHERE {{
?s a pmd:TestPiece .
?p a pmd:TensileTest .
?p pmd:input ?s .
?p pmd:characteristic ?o .
?o a pmd:PrimaryData .
FILTER regex(str(?p), "{0:s}")
}} ORDER BY ?p
""".format(example_uri)
)

list = []
try:
    ret = sparql.queryAndConvert()
    for r in ret["results"]["bindings"]:
        row = []
        for k in r.keys():
            row.append(r[k]['value'])
        list.append(row)

except Exception as e:
    print(e)

df = pd.DataFrame(list, columns =['Process (uri)', 'Metadata'])
print(tabulate(df, headers=["Metadata", "Process (uri)"], tablefmt='psql', showindex=False))

- get the url to the raw data file

In [ ]:
sparql.setQuery("""
PREFIX base: <https://w3id.org/pmd/co/>
PREFIX csvw: <http://www.w3.org/ns/csvw#>
SELECT ?url ?p
WHERE {{
    ?p a base:TensileTest .
    ?p base:characteristic ?dataset .
    ?dataset a base:Dataset .
    ?dataset base:resource ?table .
    ?table a csvw:Table .
    ?table csvw:url ?url .
    FILTER regex(str(?p), "{0:s}")
}}
ORDER BY ?p
""".format(example_uri)
)

res = sparql.queryAndConvert()
example_url = res["results"]["bindings"][0]["url"]["value"]
print(example_url)

This is not actually the raw csv bit a html website. Do some replacements:

In [ ]:
def do_replacements(urls, orign_strings=["github.com", "tree/"], replacements=["raw.githubusercontent.com", ""]):
    for orign_str, repl in zip(orign_strings, replacements):
        urls = [new.replace(orign_str, repl) for new in urls]
    return(urls)

example_url_raw = do_replacements([example_url])[0]
print(example_url_raw)

- Get all columns for one dataset (here: Zd2) (Units of the data columns can be found here)

In [ ]:
sparql.setQuery("""
SELECT DISTINCT ?part (count(?mid) AS ?column_num) ?url ?type ?unit
WHERE {{
    ?table <http://www.w3.org/ns/csvw#url> ?url .
    ?table <http://www.w3.org/ns/csvw#tableSchema> ?schema .
    ?schema <http://www.w3.org/ns/csvw#column>/rdf:rest* ?mid .
    ?mid rdf:rest* ?node .
    ?node rdf:first ?part .
    ?part a ?type .
    ?part <https://w3id.org/pmd/co/unit> ?unit .
    FILTER (?type!=<http://www.w3.org/ns/csvw#Column>)
    FILTER (str(?url)='{0:s}')
}}
GROUP BY ?node ?part ?type ?url ?unit ORDER BY ?url ?column_num
""".format(example_url)
)

liste=[]
try:
    ret = sparql.queryAndConvert()
    for r in ret["results"]["bindings"]:
        row = []
        for k in r.keys():
            # print(k, r[k]['value'])
            row.append(r[k]['value'])
        liste.append(row)
        # print ()

except Exception as e:
    print(e)

df = pd.DataFrame(liste)
print(tabulate(df, headers=["Column Index", "Node" , "Type" , "Unit" , "URL"], tablefmt='psql', showindex=False))

- query the metadata "crossectionArea_S0" (value & unit) for the example uri:

In [ ]:
sparql.setQuery("""
PREFIX pmd: <https://w3id.org/pmd/co/>
SELECT distinct ?p ?S0val ?unit
WHERE {{
    ?s a pmd:TestPiece .
    ?p a pmd:TensileTest .
    ?p pmd:input ?s .
    ?p pmd:characteristic ?output .
    ?output a pmd:CrossSectionArea .
    ?output pmd:value ?S0val .
    ?output pmd:unit ?unit .
    FILTER regex(str(?output), "S0")
    FILTER regex(str(?p), "{0:s}")
}}
ORDER BY ?p
""".format(example_uri)
)

res = sparql.queryAndConvert()
example_csa = float(res["results"]["bindings"][0]["S0val"]["value"])
example_csa_unit = res["results"]["bindings"][0]["unit"]["value"]
print(str(example_csa))
print(example_csa_unit)

## calculate Youngs modulus $E$ and offset yield $R_{p0.2}$ in an automated fashion using a `pyiron` workflow

- Import relevant modules

In [ ]:
from pyiron_tensile_test import Project
import matplotlib.pylab as plt
%matplotlib inline
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

- define SPARQL endpoint and access-token

In [ ]:
sparqlendpoint_url = SPARQL_ENDPOINT
access_token = TOKEN

- getting all uris

In [ ]:
sparql = SPARQLWrapper(sparqlendpoint_url)
sparql.setReturnFormat("json")
sparql.addCustomHttpHeader(httpHeaderName="Authorization", httpHeaderValue=access_token)

sparql.setQuery("""
PREFIX pmd: <https://w3id.org/pmd/co/>
SELECT distinct ?p
WHERE {
?s a pmd:TestPiece .
?p a pmd:TensileTest .
} ORDER BY ?p
"""
)

ret = sparql.queryAndConvert()
uris = []
for r in ret["results"]["bindings"]:
    uris.append(r['p']['value'])

### Analizing a single dataset

In [ ]:
pr = Project('tensiletest_analysis')
pr.remove_jobs(silently=True)
job = pr.create.job.TensileJob(job_name='tensile_job_single_run', delete_existing_job=True)

job.endpoint = sparqlendpoint_url
job.access_token = access_token
job.uri = uris[1]

In [ ]:
job.query_cross_section_area()
job.query_raw_data_url()
job.load_force_and_strain()
job.calc_stress()

job.run()

E = job.output.elastic_modulus
Rp02 = job.output.Rp02

fname = job.input.raw_data_url[-7:]
testname = job.uri.replace('https://w3id.org/pmd/ao/tte/pmdao-tto-tt-', '')
testname = testname.replace('_process', '')
testname = testname.replace('-', '_')

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(job.input.strain, job.input.stress/1e+09, label=testname+" ("+fname+r"), E={0:.2f} GPa, $R_{{p0.2}}$={1:.2f} MPa".format(E/1e+09, Rp02/1e+06))
ax.set_ylim(bottom=-0.01)
ax.set_xlim(left=-0.001)
ax.set_xlabel('Strain')
ax.set_ylabel('Stress / GPa')

# plot lines indicating the calculation of E and Rp0.2
ax.plot(job.input.strain, E*job.input.strain/1e+09, label=None, color='k', dashes=(10.,2.5))
ax.plot(job.input.strain, E*(job.input.strain-0.002)/1e+09, label=None, color='g', dashes=(10.,2.5))
ax.legend()

- query a list of the raw-data urls

In [ ]:
pr = Project('tensiletest_batch_analysis')
pr.remove_jobs(silently=True)

In [ ]:
fig, ax = plt.subplots(1,1)
for uri in uris:
    # generate a nice identifier for the jobs - not actually neccessary
    jobstr = uri.replace('https://w3id.org/pmd/ao/tte/pmdao-tto-tt-', '')
    jobstr = jobstr.replace('_process', '')
    jobstr = jobstr.replace('-', '_') # pyiron doesn't like hyphens

    job = pr.create.job.TensileJob(job_name='tensile_job_'+jobstr, delete_existing_job=True)
    
    job.endpoint = sparqlendpoint_url
    job.access_token = access_token
    job.uri = uri

    job.query_cross_section_area()
    job.query_raw_data_url()
    job.load_force_and_strain()
    job.calc_stress()

    # calculate E and Rp02
    job.run()
    
    E = job.output.elastic_modulus
    Rp02 = job.output.Rp02

    fname = job.input.raw_data_url[-7:]
    
    ax.plot(job.input.strain, job.input.stress/1e+09, label=jobstr+" ("+fname+r"), E={0:.2f} GPa, $R_{{p0.2}}$={1:.2f} MPa".format(E/1e+09, Rp02/1e+06))
    ax.set_ylim(bottom=-0.01)
    ax.set_xlim(left=-0.001)
    ax.set_xlabel('Strain')
    ax.set_ylabel('Stress / GPa')
    ax.legend()
#pr.job_table()